# Doing OLS our own way

In [1]:
import numpy as np

## Simulate some data

In [2]:
data_properties = {
    'n_obs': 10000,
    'beta': np.array([0.25, 0.75, -2.25]),
    'sigma': 5
}

In [31]:
def gen_data(dgp):

    np.random.seed(1234567890)
    
    n_obs = dgp['n_obs']
    beta  = dgp['beta']
    sigma = dgp['sigma']
    n_x   = beta.shape[0] - 1

    # reshape beta so we dont f* up
    beta = beta.reshape(beta.shape[0], 1)
    # 
    print('n of xs:', n_x)

    # make explanatory vars
    iota = np.ones(n_obs).reshape(n_obs,1)
    expl_var = np.random.random(size = (n_obs, n_x))

    X = np.column_stack(tup = (iota, expl_var))

    u = np.random.normal(loc = 0, scale = sigma, size = n_obs).reshape(n_obs, 1)

    # generate y
    y = X @ beta + u

    X_no_cons = X[:, 1:]

    return (y, X_no_cons)

In [34]:
y, X = gen_data(dgp = data_properties)

n of xs: 2


In [15]:
y.shape

(10000, 1)

In [16]:
X.shape

(10000, 2)

## Estimate OLS coefficients

In [22]:
def compute_ols(expl_var, dep_var, add_constant = True):

    # type check stuff
    assert type(expl_var) == np.ndarray, "Please give me a numpy array"
    assert type(dep_var)  == np.ndarray
    assert type(add_constant) == bool

    assert expl_var.shape[0] == dep_var.shape[0]
    n_obs = expl_var.shape[0]

    # add a constant?
    if add_constant == True:
        iota = np.ones(n_obs).reshape(n_obs, 1)
        expl_var = np.column_stack(tup = (iota, expl_var)) 

    XX = expl_var.T.dot(expl_var)
    Xy = expl_var.T.dot(dep_var)
    XX_inv = np.linalg.inv(XX)

    beta_hat = XX_inv.dot(Xy)

    assert beta_hat.shape[0] == expl_var.shape[1]

    return beta_hat

In [35]:
compute_ols(dep_var = y, expl_var = X, add_constant= True)

array([[ 0.28330314],
       [ 0.78398326],
       [-2.31251796]])

In [41]:
def get_residuals(expl_var, dep_var, add_constant = True):
    # do asserting

    if add_constant == True:
        iota = np.ones(dep_var.shape[0]).reshape(dep_var.shape[0])
        expl_var = np.column_stack(tup = (iota, expl_var))
    # run ols
    beta_hat = compute_ols(dep_var = dep_var, expl_var = expl_var, add_constant = False)
    u_hat = dep_var - expl_var.dot(beta_hat)

    return u_hat

In [42]:
get_residuals(expl_var= X, dep_var= y)

array([[  1.31826161],
       [-12.26057197],
       [ -2.61458708],
       ...,
       [ 12.0708953 ],
       [  2.79529088],
       [ -1.69472898]])

In [51]:
def compute_sigma2(expl_var, dep_var, add_constant = True):

    u = get_residuals(expl_var= expl_var, dep_var= dep_var)
    #print(u.shape)
    uu = u.T.dot(u)
    df = expl_var.shape[0] - expl_var.shape[1]

    sigma2 = uu / df
    return sigma2


In [52]:
compute_sigma2(expl_var=X, dep_var= y)

array([[25.5014485]])

In [61]:
def compute_vcov(expl_var, dep_var, constant = True):
    n_obs = expl_var.shape[0]
    
    sigma2 = compute_sigma2(expl_var = expl_var, dep_var = dep_var)
    #print(sigma2)

    if constant == True:
        iota = np.ones(n_obs).reshape(n_obs, 1)
        expl_var = np.column_stack(tup = (iota, expl_var))
    
    XX = expl_var.T.dot(expl_var)
    XX_inv = np.linalg.inv(XX)

    vcov_mat = sigma2 * XX_inv
    std_errors = np.sqrt(np.diag(vcov_mat))


    return std_errors, vcov_mat

In [60]:
my_errors, my_vcov = compute_vcov(expl_var=X, dep_var = y)

[[25.5014485]]


In [62]:
my_errors

array([0.13476729, 0.17435964, 0.17513707])